# Vocoder Hyperparameter Optimization

Multi-objective Bayesian optimization using Optuna's ask-and-tell interface.

**Objectives (all minimized):** SC, LogMag, Waveform L1, Mel Spec L1

**Hyperparameters:**
- Learning rate: 1e-4 to 1e-2 (log scale)
- Batch size: 32, 64, or 128
- Weight decay: 1e-6 to 1e-2 (log scale)

In [ ]:
# Run once: pip install optuna optuna-dashboard

In [5]:
import optuna

DB_PATH = "sqlite:///vocoder_study.db"
STUDY_NAME = "vocoder_v14"

study = optuna.create_study(
    study_name=STUDY_NAME,
    storage=DB_PATH,
    directions=["minimize"],  # composite score incorporating reconstruction losses and run viability
    load_if_exists=True
)

print(f"Study '{STUDY_NAME}' loaded/created with {len(study.trials)} existing trials")

[I 2025-11-29 00:17:55,400] A new study created in RDB with name: vocoder_v14


Study 'vocoder_v14' loaded/created with 0 existing trials


## Load Historical Trials

Edit the list below with your existing trial data, then run the cell once.

In [ ]:
historical_trials = [
    # {"params": {"lr": 1e-4, "batch_size": 32, "weight_decay": 1e-5}, "values": [SC, MagLoss, WaveL1, MelRecon, ComplexSTFT, GANLoss, FMLoss, DiscLoss]},
]

if historical_trials:
    for trial_data in historical_trials:
        study.enqueue_trial(trial_data["params"])
        trial = study.ask()
        if "values" in trial_data:
            study.tell(trial, trial_data["values"])
        else:
            study.tell(trial, state=optuna.trial.TrialState.FAIL)
    print(f"Loaded {len(historical_trials)} historical trials")
else:
    print("No historical trials to load (list is empty)")

Loaded 2 historical trials


## Get Next Suggestion

Run this cell to get parameters for your next training run.

In [29]:
trial = study.ask()

# Define parameter spaces (only needed for generating new suggestions)
batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
base_lr = trial.suggest_float("base_lr", 1e-5, 1e-3, log=True)
lr = base_lr * (batch_size / 32.0)
discriminator_lr_ratio = trial.suggest_float("discriminator_lr_ratio", 0.5, 2.0)
discriminator_lr = lr * discriminator_lr_ratio
gan_start_step = trial.suggest_float("gan_start_step", 0.3, 0.7)
weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)

sc_raw = trial.suggest_float("sc_raw", 0.1, 2.0)
mag_raw = trial.suggest_float("mag_raw", 0.1, 5.0)
wave_raw = trial.suggest_float("wave_raw", 0.1, 2.0)
mel_raw = trial.suggest_float("mel_raw", 0.1, 2.0)
complex_raw = trial.suggest_float("complex_raw", 0.1, 3.0)

total_raw = sc_raw + mag_raw + wave_raw + mel_raw + complex_raw
target_total = 5.0

sc_loss_weight = sc_raw / total_raw * target_total
mag_loss_weight = mag_raw / total_raw * target_total
wave_l1_loss_weight = wave_raw / total_raw * target_total
mel_recon_loss_weight = mel_raw / total_raw * target_total
complex_stft_loss_weight = complex_raw / total_raw * target_total

gan_loss_weight = trial.suggest_float("gan_loss_weight", 0.1, 1.0)
fm_ratio = trial.suggest_float("fm_ratio", 1.0, 5.0)
feature_matching_loss_weight = gan_loss_weight * fm_ratio

print(f"--learning_rate {lr} --weight_decay {weight_decay} --sc_loss_weight {sc_loss_weight} --mag_loss_weight {mag_loss_weight} --wave_l1_loss_weight {wave_l1_loss_weight} --mel_recon_loss_weight {mel_recon_loss_weight} --complex_stft_loss_weight {complex_stft_loss_weight} --gan_loss_weight {gan_loss_weight} --feature_matching_loss_weight {feature_matching_loss_weight} --discriminator_lr {discriminator_lr}")
print('{"params": {"lr": '+str(lr)+', "batch_size": '+str(batch_size)+', "weight_decay": '+str(weight_decay)+', "sc_loss_weight": '+str(sc_loss_weight)+', "mag_loss_weight": '+str(mag_loss_weight)+', "wave_l1_loss_weight": '+str(wave_l1_loss_weight)+', "mel_recon_loss_weight": '+str(mel_recon_loss_weight)+', "complex_stft_loss_weight": '+str(complex_stft_loss_weight)+ ', "gan_loss_weight": '+str(gan_loss_weight)+', "feature_matching_loss_weight": '+str(feature_matching_loss_weight)+', "discriminator_lr": '+str(discriminator_lr)+', "gan_start_step": '+str(gan_start_step)+'}}')

--learning_rate 0.0007105249615645953 --weight_decay 0.00012551252892016514 --sc_loss_weight 0.991773834269867 --mag_loss_weight 1.8910770223024658 --wave_l1_loss_weight 0.1930552779587998 --mel_recon_loss_weight 0.28391407602588115 --complex_stft_loss_weight 1.6401797894429857 --gan_loss_weight 0.5324566225883319 --feature_matching_loss_weight 2.5544041807691356 --discriminator_lr 0.0009555942265828252
{"params": {"lr": 0.0007105249615645953, "batch_size": 64, "weight_decay": 0.00012551252892016514, "sc_loss_weight": 0.991773834269867, "mag_loss_weight": 1.8910770223024658, "wave_l1_loss_weight": 0.1930552779587998, "mel_recon_loss_weight": 0.28391407602588115, "complex_stft_loss_weight": 1.6401797894429857, "gan_loss_weight": 0.5324566225883319, "feature_matching_loss_weight": 2.5544041807691356, "discriminator_lr": 0.0009555942265828252, "gan_start_step": 0.4946179393972899}}


In [ ]:
# convert gan metric to step count
total_examples = 1_100_000
num_epochs = 3
total_steps = (total_examples / float(batch_size)) * num_epochs
gan_start_step_i = int(gan_start_step * total_steps)
print(f"total_steps: {total_steps} --gan_start_step {gan_start_step_i}")

total_steps: 51562.5 --gan_start_step 25503


## Compute Result Scores

In [19]:
def compute_trial_score(min_loss, steps_completed, total_steps):
    progress = steps_completed / total_steps
    # Penalty for dying early: ranges from 0 (finished) to some max (died immediately)
    early_death_penalty = (1.0 - progress) * min_loss  # Scale penalty by loss magnitude
    return min_loss + early_death_penalty

# Fill these in with your results
sc_loss = 0.8789       # Spectral Convergence
logmag_loss = 2.875   # Log Magnitude
wave_l1 = 0.01202       # Waveform L1
mel_l1 = 3.016        # Mel Spectrogram L1
complex_stft_loss = 0.1232  # Complex STFT Loss
steps_completed = 36600  # e.g., 8000
run_failed = False # if the run failed/crashed

trial_score = compute_trial_score(min_loss=sc_loss+logmag_loss+wave_l1+mel_l1+complex_stft_loss, steps_completed=steps_completed, total_steps=total_steps)
print(f"Trial Score: {trial_score}")

Trial Score: 11.359550138181817


## Record Results

After training completes, fill in your 4 loss values and run this cell.

In [ ]:
# Record the result
if run_failed:
    study.tell(trial, state=optuna.trial.TrialState.FAIL)
    print(f"Trial #{trial.number} marked as FAILED")
elif None in [sc_loss, logmag_loss, wave_l1, mel_l1, complex_stft_loss]:
    print("ERROR: Fill in all 5 loss values (or set run_failed=True)")
else:
    study.tell(trial, [trial_score])
    print(f"Trial #{trial.number} recorded!")
    print(f"  SC: {sc_loss}, LogMag: {logmag_loss}, Wave L1: {wave_l1}, Mel L1: {mel_l1}")
    print(', "values": ['+str(sc_loss)+', '+str(logmag_loss)+', '+str(wave_l1)+', '+str(mel_l1)+']')

Trial #0 marked as FAILED


## View Results

In [ ]:
print(f"Total trials: {len(study.trials)}")
print(f"Completed: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}")
print(f"Failed: {len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])}")
print()

print("Pareto-optimal trials:")
print("="*80)
for t in study.best_trials:
    print(f"Trial #{t.number}")
    print(f"  Params: lr={t.params['lr']:.6f}, batch={t.params['batch_size']}, wd={t.params['weight_decay']:.6f}")
    print(f"  Losses: SC={t.values[0]:.4f}, LogMag={t.values[1]:.4f}, WaveL1={t.values[2]:.4f}, MelL1={t.values[3]:.4f}")
    print()

## All Trials Table

In [28]:
import pandas as pd

rows = []
for t in study.trials:
    if t.state == optuna.trial.TrialState.COMPLETE:
        print(t)
        params = t.params if len(t.params.keys()) > 0 else t.system_attrs["fixed_params"]
        rows.append({
            "trial": t.number,
            "lr": params.get("lr"),
            "batch_size": params.get("batch_size"),
            "weight_decay": params.get("weight_decay"),
            "COMPOSITE": t.values[0],
        })

df = pd.DataFrame(rows)
df

FrozenTrial(number=0, state=<TrialState.COMPLETE: 1>, values=[18.440459689696965], datetime_start=datetime.datetime(2025, 11, 28, 22, 0, 13, 566849), datetime_complete=datetime.datetime(2025, 11, 28, 22, 0, 13, 577776), params={}, user_attrs={}, system_attrs={'fixed_params': {'lr': 0.0004827238320533371, 'batch_size': 32, 'weight_decay': 0.007709498399240456, 'sc_loss_weight': 0.7477651161998835, 'mag_loss_weight': 2.8307514883247005, 'wave_l1_loss_weight': 0.062224622626292095, 'mel_recon_loss_weight': 0.8897692533857661, 'complex_stft_loss_weight': 0.4694895194633577, 'gan_loss_weight': 0.5322976549045951, 'feature_matching_loss_weight': 1.8543398400790423, 'discriminator_lr': 0.00044621946542211467, 'gan_start_step': 0.40622545454545456}}, intermediate_values={}, distributions={}, trial_id=17, value=None)
FrozenTrial(number=1, state=<TrialState.COMPLETE: 1>, values=[11.359550138181817], datetime_start=datetime.datetime(2025, 11, 28, 22, 0, 13, 595058), datetime_complete=datetime.dat

,trial,lr,batch_size,weight_decay,COMPOSITE
0,0,0.000483,32,0.007709,18.44046
1,1,0.000183,32,0.000153,11.35955


## Launch Dashboard

Run this in a terminal (not in the notebook):
```bash
optuna-dashboard sqlite:///vocoder_study.db
```

Then open http://localhost:8080 in your browser.